## Import necessary libraries

In [318]:
import os
import re
import pandas as pd
import nltk
import itertools
import string
import xml.etree.ElementTree as et
import numpy as np


#use current directory as location for all files

os.chdir(r"./")

## Read dataset

In [319]:
#SemEval dataset
filename = "Restaurants_Train.xlsx"
data = pd.read_excel(filename)

In [320]:
data.head(5)

,sentences,sentence,id,text,aspectTerms,aspectTerm,term,polarity,from,to,aspectCategories,aspectCategory,category,polarity.1
0,NaN,NaN,3121.0,But the staff was so horrible to us.,NaN,NaN,staff,negative,8.0,13.0,NaN,NaN,service,negative
1,NaN,NaN,2777.0,"To be completely fair, the only redeeming fact...",NaN,NaN,food,positive,57.0,61.0,NaN,NaN,food,positive
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,anecdotes/miscellaneous,negative
3,NaN,NaN,1634.0,"The food is uniformly exceptional, with a very...",NaN,NaN,food,positive,4.0,8.0,NaN,NaN,food,positive
4,NaN,NaN,NaN,NaN,NaN,NaN,kitchen,positive,55.0,62.0,NaN,NaN,NaN,NaN


In [321]:
data.tail(5)

,sentences,sentence,id,text,aspectTerms,aspectTerm,term,polarity,from,to,aspectCategories,aspectCategory,category,polarity.1
4908,NaN,NaN,NaN,NaN,NaN,NaN,meats,neutral,99.0,104.0,NaN,NaN,NaN,NaN
4909,NaN,NaN,NaN,NaN,NaN,NaN,vegetables,neutral,114.0,124.0,NaN,NaN,NaN,NaN
4910,NaN,NaN,NaN,NaN,NaN,NaN,rice,neutral,130.0,134.0,NaN,NaN,NaN,NaN
4911,NaN,NaN,NaN,NaN,NaN,NaN,glass noodles,neutral,139.0,152.0,NaN,NaN,NaN,NaN
4912,NaN,NaN,617.0,I am going to the mid town location next.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,anecdotes/miscellaneous,neutral


## Preliminary analysis

In [322]:
keys = pd.DataFrame(data.keys())
keys.index += 1
keys = keys.rename(columns={0:"Keys"})
keys

,Keys
1,sentences
2,sentence
3,id
4,text
5,aspectTerms
6,aspectTerm
7,term
8,polarity
9,from
10,to


In [323]:
reviews = data["text"].dropna()
print("Number of Reviews:", len(reviews))

Number of Reviews: 3044


In [324]:
ex_asp = pd.unique(data["category"].dropna())
print("Explicit Aspects:", ", ".join(ex_asp))

Explicit Aspects: service, food, anecdotes/miscellaneous, price, ambience


In [325]:
ex_asp_freq_table = data.groupby("category").size().reset_index()
ex_asp_freq_table = ex_asp_freq_table.rename(columns={"category":"Ex.Asp", 0:"Num"})
ex_asp_freq_table.index +=1
ex_asp_freq_table

,Ex.Asp,Num
1,ambience,432
2,anecdotes/miscellaneous,1133
3,food,1233
4,price,319
5,service,597


In [326]:
asp_polarity_freq_table = data.groupby(["category", "polarity.1"]).size().reset_index()
asp_polarity_freq_table = asp_polarity_freq_table.rename(columns={"category":"Ex.Asp", "polarity.1":"Polarity", 0:"Num"})
asp_polarity_freq_table.index +=1
asp_polarity_freq_table

,Ex.Asp,Polarity,Num
1,ambience,conflict,47
2,ambience,negative,98
3,ambience,neutral,24
4,ambience,positive,263
5,anecdotes/miscellaneous,conflict,30
6,anecdotes/miscellaneous,negative,199
7,anecdotes/miscellaneous,neutral,357
8,anecdotes/miscellaneous,positive,547
9,food,conflict,67
10,food,negative,209


## Data Preprocessing

In [327]:
def _removePunc(item): #replace with space
    return re.sub(r'[^\w\s]',' ',item)

In [328]:
filename = "Restaurants_Train.xml"

table = []
row = [np.NaN] * 7

for event, node in et.iterparse(filename, events=('start', 'end')):
    
    if node.tag == "text":
        row[0] = node.text
    elif node.tag == "aspectTerms" and event == "start":
        row[1] = []
        row[2] = []
        row[3] = []
        row[4] = []
    elif node.tag == "aspectTerm" and event == "start":
        row[1].append(_removePunc(node.attrib.get("term")))
        row[2].append(node.attrib.get("polarity"))
        row[3].append(int(node.attrib.get("from")))
        row[4].append(int(node.attrib.get("to")))
    elif node.tag == "aspectCategories" and event == "start":
        row[5] = []
        row[6] = []
    elif node.tag == "aspectCategory" and event == "start":
        row[5].append(node.attrib.get("category"))
        row[6].append(node.attrib.get("polarity"))
    elif node.tag == "aspectCategories" and event == "end":
        table.append(row)
        row = [np.NaN] * 7

dfcols = ['review', 'term', 'termPolarity', 'startIndex', 'endIndex','aspect', 'aspectPolarity']
clean_data = pd.DataFrame(table, columns=dfcols)

In [329]:
clean_data.head(5)

,review,term,termPolarity,startIndex,endIndex,aspect,aspectPolarity
0,But the staff was so horrible to us.,[staff],[negative],[8],[13],[service],[negative]
1,"To be completely fair, the only redeeming fact...",[food],[positive],[57],[61],"[food, anecdotes/miscellaneous]","[positive, negative]"
2,"The food is uniformly exceptional, with a very...","[food, kitchen, menu]","[positive, positive, neutral]","[4, 55, 141]","[8, 62, 145]",[food],[positive]
3,Where Gabriela personaly greets you and recomm...,NaN,NaN,NaN,NaN,[service],[positive]
4,"For those that go once and don't enjoy it, all...",NaN,NaN,NaN,NaN,[anecdotes/miscellaneous],[positive]


In [330]:
clean_data.loc[2, "review"]

"The food is uniformly exceptional, with a very capable kitchen which will proudly whip up whatever you feel like eating, whether it's on the menu or not."

In [331]:
clean_data.tail(5)

,review,term,termPolarity,startIndex,endIndex,aspect,aspectPolarity
3039,But that is highly forgivable.,NaN,NaN,NaN,NaN,[anecdotes/miscellaneous],[positive]
3040,"From the appetizers we ate, the dim sum and ot...","[appetizers, dim sum, foods, food]","[positive, positive, positive, positive]","[9, 32, 61, 103]","[19, 39, 66, 107]",[food],[positive]
3041,"When we arrived at 6:00 PM, the restaurant was...",NaN,NaN,NaN,NaN,[anecdotes/miscellaneous],[neutral]
3042,Each table has a pot of boiling water sunken i...,"[table, pot of boiling water, meats, vegetable...","[neutral, neutral, neutral, neutral, neutral, ...","[5, 17, 99, 114, 130, 139]","[10, 37, 104, 124, 134, 152]",[food],[neutral]
3043,I am going to the mid town location next.,NaN,NaN,NaN,NaN,[anecdotes/miscellaneous],[neutral]


#### Encode labels into a categorical vector
Create a function to find the word index after splitting the review into list

In [332]:
# index a term in the string list (with customization)
def _index(str, term):
    for i in range(len(str)):
        if term in str[i]: # More lenient as not need for exactly the same word
            return i
    raise("Word not found")

In [333]:
def _findWordIndex(s, terms):
    #return a vector of len 65, with 0 representing not a aspect term and 1 representing an aspect term
    
    answer = [0]*65
    # Terminate if no idx given (term is NaN, NaN is a float)
    if isinstance(terms, float):
        #
        return answer

    # Temp vars
    new_str = _removePunc(s).split()
    word_idx = []
    
    for term in terms:
        for subterm in term.split():
            word_idx.append(_index(new_str, subterm))
            
    for index in word_idx:
        answer[index] = 1
            
    return answer

Make a new column for categorical terms

In [334]:
# Make new column for categorical terms
categoricalTerms = []
terms = clean_data["term"]
reviews = clean_data["review"]

for i in range(clean_data.shape[0]):
    categoricalTerms.append(_findWordIndex(reviews[i], terms[i]))

clean_data = clean_data.assign(categoricalTerms=pd.Series(categoricalTerms))

In [335]:
clean_data.head(5)

,review,term,termPolarity,startIndex,endIndex,aspect,aspectPolarity,categoricalTerms
0,But the staff was so horrible to us.,[staff],[negative],[8],[13],[service],[negative],"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"To be completely fair, the only redeeming fact...",[food],[positive],[57],[61],"[food, anecdotes/miscellaneous]","[positive, negative]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
2,"The food is uniformly exceptional, with a very...","[food, kitchen, menu]","[positive, positive, neutral]","[4, 55, 141]","[8, 62, 145]",[food],[positive],"[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
3,Where Gabriela personaly greets you and recomm...,NaN,NaN,NaN,NaN,[service],[positive],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"For those that go once and don't enjoy it, all...",NaN,NaN,NaN,NaN,[anecdotes/miscellaneous],[positive],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


#### Testing the correctness of _findWordIndex function
Activate the block by turning it to "Code" if needed

# -----------
# Testing 123
# -----------
#### Loading necessary libraries

In [336]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from nltk.corpus import stopwords

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D
from keras.layers.core import Reshape, Flatten, Activation
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam
from keras.models import Model
from keras import regularizers

import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

#### Preprocess the data

#### Split data into training, testing, validation sets

In [337]:
reviews = clean_data['review']
labels = clean_data['categoricalTerms']
### rmb to remove conflict
x_train, x_test, y_train, y_test = train_test_split(reviews, labels, test_size = 0.4, random_state=999)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size = 0.5, random_state=999)


#### Tokenize text of the training data with keras preprocessing text function

In [338]:
# Set maximum number of words to be embedded
NUM_WORDS = 100000

# Set maximum length of a sentence
MAX_LEN = 65

# Define/Load Tokenize text function
tokenizer = Tokenizer(num_words=NUM_WORDS, filters=string.punctuation)

# Fit the function on the text
tokenizer.fit_on_texts(x_train)

# Count number of unique tokens
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 3417 unique tokens.


In [339]:
# Convert train and val to sequence
# sequences_train = tokenizer.texts_to_sequences(x_train)
# sequences_valid= tokenizer.texts_to_sequences(x_val)


In [350]:
x_train.head(5)
x_train.shape

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [341]:
y_train.head(5)

1224    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
12      [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...
2724    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
461     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
744     [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, ...
Name: categoricalTerms, dtype: object

In [342]:
# Limit size of train/val to 65 and pad the sequence
x_train = pad_sequences(sequences_train,maxlen=MAX_LEN)
x_val = pad_sequences(sequences_valid,maxlen=x_train.shape[1])

# Use multilabelbinarizer to encode categorical terms into one-hot vector
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(y_train)
y_val = mlb.fit_transform(y_val)
print(y_train[4])

# Printing shape
print('Shape of X train and X validation tensor:', x_train.shape,x_val.shape)
print('Shape of label train and validation tensor:', y_train.shape,y_val.shape)

[1 1]
Shape of X train and X validation tensor: (1826, 65) (609, 65)
Shape of label train and validation tensor: (1826, 2) (609, 2)


## Word Embedding
#### Using pretrained Word2Vec model from Google or Amazon
#### Create 300-dimensional vectors

In [343]:
#word_vectors = KeyedVectors.load_word2vec_format('./AmazonWE/sentic2vec.csv')
word_vectors = pd.read_csv('./sentic2vec.csv', encoding = "ISO-8859-1", header=None)

In [344]:
word_vectors.head(5)

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
0,nycsamsung,0.017007,-0.010965,0.034439,-0.003403,0.001669,-0.016725,-0.009386,-0.014179,0.006383,...,-0.005171,0.027319,-0.007168,0.010212,-0.017336,0.003275,-0.001003,0.011749,-0.004390,0.012654
1,localizer,0.038885,0.059859,-0.051594,0.021452,0.001800,-0.010394,-0.010728,0.096014,0.012083,...,-0.087075,-0.005560,-0.016801,0.044161,-0.010343,0.055683,-0.026794,-0.046406,-0.002826,0.046562
2,gah,0.000715,-0.001516,-0.000800,-0.000104,-0.000850,-0.000913,0.000452,-0.001184,-0.000177,...,-0.000289,0.000471,0.001467,-0.000577,0.000112,-0.000166,0.001002,0.000662,-0.000665,-0.001045
3,hasselfree,0.014361,-0.000866,-0.035235,0.024500,0.034341,-0.010994,-0.012281,0.020808,-0.034909,...,-0.010619,0.013001,-0.021808,0.006658,-0.023384,0.019640,0.028458,-0.030321,-0.042621,-0.011562
4,prendeel,0.025254,0.015678,-0.035391,-0.012458,0.045409,-0.019798,-0.004205,0.014728,-0.012590,...,-0.015791,0.002601,-0.038128,0.007929,0.001784,0.023809,-0.023151,-0.005321,-0.017844,0.033579


In [345]:
EMBEDDING_DIM=300
vocabulary_size=min(len(word_index)+1,(NUM_WORDS))

window_size = 5

embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM*window_size))

for word, i in word_index.items():
    if i < vocabulary_size:
        embedding_vector = word_vectors.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
#         else:
#             vec = np.zeros(EMBEDDING_DIM)
#             embedding_matrix[i] = vec

In [346]:
del(word_vectors)

In [347]:
# Define embedding function using the embedding_matrix
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True,
                            input_length=MAX_LEN)

In [348]:
del(embedding_matrix)

#### Build CNN with EarlyStopping
What is unknown so far?
- the structure of fully-connected layer
- batch size

In [349]:
sequence_length = x_train.shape[1]
filter_sizes = [2,3]
num_filters = [100, 50]

# Create model
model = Sequential()

# Add layers
model.add(embedding_layer)
model.add(Reshape((sequence_length,EMBEDDING_DIM,1)))
model.add(Conv2D(num_filters[0],(filter_sizes[0], filter_sizes[0]), activation='tanh'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(num_filters[1],(filter_sizes[1], filter_sizes[1]), activation='tanh'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
# model.add(Dense(100, activation='relu'))
# model.add(Dense(100, activation='relu'))
model.add(Dense(MAX_LEN, activation='softmax', kernel_regularizer=regularizers.l2(0.01)))
# model.add(Activation('softmax'))

ValueError: Layer weight shape (3418, 300) not compatible with provided weight shape (3418, 1500)

In [ ]:
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

# summarize the model
print(model.summary())

In [ ]:
history = model.fit(x_train, y_train,
                    epochs=30,
                    batch_size=60,
                    validation_data=(x_val, y_val))